# E6: Conflicting Cue Experiment (FIXED VERSION)

**Paper**: A2 (Cue-Dominant Extraction Explains Length Effects)

**CRITICAL FIX**: Previous version had a bug where `Final = expr = value` format
was only partially replaced, leaving residual expressions. This version replaces
the ENTIRE final step content to ensure clean cue manipulation.

**Purpose**: Directly test whether the model prioritizes the answer CUE over the reasoning CHAIN.

**Conditions**:
| Condition | Step 1-9 | Step 10 (Cue) | Prediction |
|-----------|----------|---------------|------------|
| **A: Wrong Cue Injection** | Clean (correct reasoning) | Wrong final answer | **WRONG** (if cue-dominant) |
| **B: Correct Cue Only** | Corrupted (c=0.8) | Correct final answer | **CORRECT** (if cue-dominant) |
| **C: Control** | Clean | Correct final answer | CORRECT |

**Expected inference count**: 199 × 3 = 597

**Date**: 2026-01-03
**GLOBAL_SEED**: 20251224
**E6_SEED**: 20260103
**VERSION**: 2.0 (FIXED)

## 0. Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXPERIMENT_NAME = 'E6_conflicting_cue_v2'
EXPERIMENT_DATE = datetime.now().strftime('%Y%m%d')

BASE_DIR = '/content/drive/MyDrive/CoT_Experiment'
V3_DATA_DIR = f'{BASE_DIR}/full_experiment_v3_20251224'

SAVE_DIR = f'{BASE_DIR}/{EXPERIMENT_NAME}_{EXPERIMENT_DATE}'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f'{SAVE_DIR}/results', exist_ok=True)

print(f'Experiment: {EXPERIMENT_NAME}')
print(f'V3 data directory: {V3_DATA_DIR}')
print(f'Save directory: {SAVE_DIR}')

## 1. Install Dependencies

In [ ]:
!pip install datasets anthropic matplotlib pandas tqdm scipy -q
print('Dependencies installed.')

## 2. Configuration

In [ ]:
import hashlib
import random
import json
import re
import time
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, asdict, field
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# =============================================================================
# Global Configuration
# =============================================================================
GLOBAL_SEED = 20251224
E6_SEED = 20260103

# Corruption settings for Condition B
CORRUPTION_RATE = 0.8  # c = 0.8 for corrupted reasoning
CORRUPTION_RATIO = {'IRR': 1, 'LOC': 2, 'WRONG': 2}

# API settings
API_MAX_TOKENS_ANSWER = 256
API_RETRY_DELAY = 1.0
API_RATE_LIMIT_DELAY = 0.5

print('='*70)
print('E6: CONFLICTING CUE EXPERIMENT (FIXED VERSION 2.0)')
print('='*70)
print(f'  GLOBAL_SEED: {GLOBAL_SEED}')
print(f'  E6_SEED: {E6_SEED}')
print(f'  Corruption rate (Condition B): {CORRUPTION_RATE}')
print('='*70)

## 3. Data Structures

In [ ]:
@dataclass
class GSM8KProblem:
    index: int
    question: str
    answer_text: str
    final_answer: int

@dataclass
class CleanTrace:
    problem_index: int
    I: int  # Number of steps (L)
    steps: List[str]
    full_text: str

@dataclass
class ManipulatedTrace:
    """Trace with manipulated cue and/or reasoning"""
    problem_index: int
    L: int
    condition: str  # 'wrong_cue', 'correct_cue_only', 'control'
    reasoning_status: str  # 'clean' or 'corrupted'
    cue_status: str  # 'correct' or 'wrong'
    cue_answer: int  # The answer shown in the final step
    correct_answer: int  # The actual correct answer
    steps: List[str]
    full_text: str
    seed: int

@dataclass
class ExperimentResult:
    problem_index: int
    condition: str
    cue_answer: int
    correct_answer: int
    model_answer: Optional[int]
    is_correct: bool
    followed_cue: bool
    raw_output: str
    timestamp: str

## 4. Utility Functions

In [ ]:
def derive_seed(global_seed: int, problem_id: int, condition: str) -> int:
    key = f"{global_seed}|E6|{problem_id}|{condition}"
    h = hashlib.sha256(key.encode("utf-8")).hexdigest()
    return int(h[:8], 16)

def save_json(data: Any, filepath: str):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_json(filepath: str) -> Any:
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

## 5. Load Data

In [ ]:
# Load problems
problems_path = f'{V3_DATA_DIR}/problems_v3.json'
problems_data = load_json(problems_path)
problems = [GSM8KProblem(**p) for p in problems_data]
prob_map = {p.index: p for p in problems}
print(f'Loaded {len(problems)} problems')

# Load clean traces (L=10)
traces_path = f'{V3_DATA_DIR}/clean_traces/clean_traces_I10_v3.json'
traces_data = load_json(traces_path)
clean_traces = [CleanTrace(**t) for t in traces_data]
trace_map = {t.problem_index: t for t in clean_traces}
print(f'Loaded {len(clean_traces)} clean traces (L=10)')

## 6. Corruption Logic (for Condition B)

In [ ]:
def pick_corrupted_steps(L: int, c: float, seed: int) -> List[int]:
    """Select steps to corrupt (excluding the final step which keeps correct cue)"""
    K = int(round(c * (L - 1)))  # Exclude last step
    if K == 0:
        return []
    steps = list(range(1, L))  # Steps 1 to L-1
    rng = random.Random(seed)
    rng.shuffle(steps)
    return sorted(steps[:K])

def assign_corruption_types(corrupted_steps: List[int], seed: int) -> Dict[int, str]:
    """Assign corruption types with ratio IRR:LOC:WRONG = 1:2:2"""
    K = len(corrupted_steps)
    if K == 0:
        return {}
    n_irr = (K * 1) // 5
    n_loc = (K * 2) // 5
    n_wrong = K - n_irr - n_loc
    if n_wrong == 0 and K > 0:
        n_wrong = 1
        if n_loc > 0:
            n_loc -= 1
        elif n_irr > 0:
            n_irr -= 1
    rng = random.Random(seed + 1)
    perm = corrupted_steps[:]
    rng.shuffle(perm)
    type_map = {}
    for s in perm[:n_irr]:
        type_map[s] = "IRR"
    for s in perm[n_irr:n_irr + n_loc]:
        type_map[s] = "LOC"
    for s in perm[n_irr + n_loc:]:
        type_map[s] = "WRONG"
    return type_map

## 7. Corruption Templates

In [ ]:
IRRELEVANT_TEMPLATES = [
    "Compute an auxiliary value: aux = {a} + {b} = {result}, but it will not be used later.",
    "Compute a side quantity: aux = {a} * 2 = {result}, unrelated to the final result.",
    "Note that we can also compute aux = {a} - {b} = {result}, though this is not needed.",
]

WRONG_CONSTRAINT_TEMPLATES = [
    "Fix an intermediate condition: set {var} = {wrong_value} as a given constraint for the rest of the steps.",
    "Assume the total is {var} = {wrong_value} and proceed using this fixed value.",
]

def generate_irrelevant_step(step_num: int, seed: int) -> str:
    rng = random.Random(seed)
    a = rng.randint(2, 20)
    b = rng.randint(2, 20)
    template = rng.choice(IRRELEVANT_TEMPLATES)
    if '+' in template:
        result = a + b
    elif '*' in template:
        result = a * 2
    else:
        result = a - b
    return template.format(a=a, b=b, result=result)

def generate_local_error_step(original_step: str, seed: int) -> str:
    rng = random.Random(seed)
    numbers = re.findall(r'\d+', original_step)
    if not numbers:
        return f"Compute t = 10 * 3 = {rng.randint(28, 32)} (using the previous values)."
    original_result = int(numbers[-1])
    offset = rng.choice([-3, -2, -1, 1, 2, 3])
    wrong_result = max(0, original_result + offset)
    modified = re.sub(r'= (\d+)\.$', f'= {wrong_result}.', original_step)
    if modified == original_step:
        modified = re.sub(r'(\d+)\.$', f'{wrong_result}.', original_step)
    return modified

def generate_wrong_constraint_step(step_num: int, seed: int) -> str:
    rng = random.Random(seed)
    var = rng.choice(['x', 'total', 'result', 'n'])
    wrong_value = rng.randint(10, 100)
    template = rng.choice(WRONG_CONSTRAINT_TEMPLATES)
    return template.format(var=var, wrong_value=wrong_value)

## 8. CRITICAL FIX: Final Step Manipulation

**BUG IN PREVIOUS VERSION**: Partial replacement of `Final = expr` left residual expressions.

**FIX**: Replace the ENTIRE final step content with a clean cue.

In [ ]:
def generate_wrong_answer(correct_answer: int, seed: int) -> int:
    """
    Generate a wrong answer that is plausible but clearly different.
    """
    rng = random.Random(seed)
    
    # Generate offset based on magnitude
    magnitude = max(1, abs(correct_answer))
    
    if magnitude < 10:
        offset = rng.choice([-5, -4, -3, -2, 2, 3, 4, 5])
    elif magnitude < 100:
        offset = rng.choice([-20, -15, -10, 10, 15, 20])
    elif magnitude < 1000:
        offset = rng.choice([-100, -50, 50, 100, 150])
    else:
        # For large numbers, use percentage-based offset
        pct = rng.choice([0.1, 0.15, 0.2, -0.1, -0.15, -0.2])
        offset = int(correct_answer * pct)
    
    wrong_answer = correct_answer + offset
    
    # Ensure it's different and positive if original was positive
    if wrong_answer == correct_answer:
        wrong_answer = correct_answer + (10 if correct_answer >= 0 else -10)
    if correct_answer > 0 and wrong_answer <= 0:
        wrong_answer = abs(wrong_answer) + 1
    
    return wrong_answer

def create_clean_final_step(answer: int) -> str:
    """
    Create a clean final step with ONLY the cue.
    
    CRITICAL: This replaces the ENTIRE final step content to avoid
    any residual expressions from the original step.
    """
    return f"Therefore, the final answer is Final = {answer}."

# Test the fix
print("=== TESTING FINAL STEP CREATION ===")
print(f"Correct answer 70000: {create_clean_final_step(70000)}")
print(f"Wrong answer 80500: {create_clean_final_step(80500)}")
print()
print("This ensures NO residual expressions like 'Final = 80500 - 130000'")

## 9. Trace Creation for Each Condition

In [ ]:
def create_condition_a_trace(clean_trace: CleanTrace, correct_answer: int, seed: int) -> ManipulatedTrace:
    """
    Condition A: Wrong Cue + Clean Reasoning
    - Steps 1-9: Keep clean (correct reasoning)
    - Step 10: REPLACE ENTIRELY with wrong cue
    """
    wrong_answer = generate_wrong_answer(correct_answer, seed)
    
    # Keep steps 1-9 clean, replace step 10 ENTIRELY
    new_steps = clean_trace.steps[:-1] + [create_clean_final_step(wrong_answer)]
    
    # Build full text
    lines = ['[[COT_START]]']
    for i, content in enumerate(new_steps):
        lines.append(f'Step {i+1}: {content}')
    lines.append('[[COT_END]]')
    full_text = '\n'.join(lines)
    
    return ManipulatedTrace(
        problem_index=clean_trace.problem_index,
        L=clean_trace.I,
        condition='wrong_cue',
        reasoning_status='clean',
        cue_status='wrong',
        cue_answer=wrong_answer,
        correct_answer=correct_answer,
        steps=new_steps,
        full_text=full_text,
        seed=seed
    )

def create_condition_b_trace(clean_trace: CleanTrace, correct_answer: int, seed: int) -> ManipulatedTrace:
    """
    Condition B: Correct Cue + Corrupted Reasoning
    - Steps 1-9: Corrupt with c=0.8
    - Step 10: REPLACE ENTIRELY with correct cue
    """
    L = clean_trace.I
    
    # Corrupt steps 1 to L-1
    corrupted_steps = pick_corrupted_steps(L, CORRUPTION_RATE, seed)
    corruption_types = assign_corruption_types(corrupted_steps, seed)
    
    new_steps = []
    for i, step_content in enumerate(clean_trace.steps[:-1]):  # All but last
        step_num = i + 1
        if step_num in corrupted_steps:
            ctype = corruption_types[step_num]
            step_seed = seed + step_num * 1000
            if ctype == 'IRR':
                new_content = generate_irrelevant_step(step_num, step_seed)
            elif ctype == 'LOC':
                new_content = generate_local_error_step(step_content, step_seed)
            else:
                new_content = generate_wrong_constraint_step(step_num, step_seed)
            new_steps.append(new_content)
        else:
            new_steps.append(step_content)
    
    # Add clean final step with CORRECT cue
    new_steps.append(create_clean_final_step(correct_answer))
    
    # Build full text
    lines = ['[[COT_START]]']
    for i, content in enumerate(new_steps):
        lines.append(f'Step {i+1}: {content}')
    lines.append('[[COT_END]]')
    full_text = '\n'.join(lines)
    
    return ManipulatedTrace(
        problem_index=clean_trace.problem_index,
        L=L,
        condition='correct_cue_only',
        reasoning_status='corrupted',
        cue_status='correct',
        cue_answer=correct_answer,
        correct_answer=correct_answer,
        steps=new_steps,
        full_text=full_text,
        seed=seed
    )

def create_condition_c_trace(clean_trace: CleanTrace, correct_answer: int, seed: int) -> ManipulatedTrace:
    """
    Condition C: Control (Clean Reasoning + Correct Cue)
    - Steps 1-9: Keep clean
    - Step 10: REPLACE ENTIRELY with correct cue (for consistency)
    """
    # Keep steps 1-9, replace step 10 with clean correct cue
    new_steps = clean_trace.steps[:-1] + [create_clean_final_step(correct_answer)]
    
    # Build full text
    lines = ['[[COT_START]]']
    for i, content in enumerate(new_steps):
        lines.append(f'Step {i+1}: {content}')
    lines.append('[[COT_END]]')
    full_text = '\n'.join(lines)
    
    return ManipulatedTrace(
        problem_index=clean_trace.problem_index,
        L=clean_trace.I,
        condition='control',
        reasoning_status='clean',
        cue_status='correct',
        cue_answer=correct_answer,
        correct_answer=correct_answer,
        steps=new_steps,
        full_text=full_text,
        seed=seed
    )

In [ ]:
# Verify the fix with a sample
sample_trace = clean_traces[0]
sample_problem = prob_map[sample_trace.problem_index]
sample_seed = derive_seed(E6_SEED, sample_trace.problem_index, 'test')

print("=== VERIFICATION: Original vs Fixed ===")
print(f"Problem index: {sample_trace.problem_index}")
print(f"Correct answer: {sample_problem.final_answer}")
print()
print(f"Original Step 10: {sample_trace.steps[-1]}")
print()

# Test Condition A
cond_a = create_condition_a_trace(sample_trace, sample_problem.final_answer, sample_seed)
print(f"Condition A (wrong cue) Step 10: {cond_a.steps[-1]}")
print(f"  Cue answer: {cond_a.cue_answer}, Correct: {cond_a.correct_answer}")
print()

# Test Condition B
cond_b = create_condition_b_trace(sample_trace, sample_problem.final_answer, sample_seed)
print(f"Condition B (correct cue) Step 10: {cond_b.steps[-1]}")
print(f"  Cue answer: {cond_b.cue_answer}, Correct: {cond_b.correct_answer}")
print()

# Test Condition C
cond_c = create_condition_c_trace(sample_trace, sample_problem.final_answer, sample_seed)
print(f"Condition C (control) Step 10: {cond_c.steps[-1]}")
print(f"  Cue answer: {cond_c.cue_answer}, Correct: {cond_c.correct_answer}")

# Confirm no residual expressions
print()
print("=== CONFIRMATION ===")
for name, trace in [('A', cond_a), ('B', cond_b), ('C', cond_c)]:
    step10 = trace.steps[-1]
    has_operator = any(op in step10 for op in [' - ', ' + ', ' × ', ' ÷ ', ' * ', ' / '])
    print(f"Condition {name}: Has operator in Step 10? {has_operator}")
    if has_operator:
        print(f"  WARNING: {step10}")

## 10. API Setup

In [ ]:
from getpass import getpass

ANTHROPIC_API_KEY = getpass('Enter Anthropic API Key: ')
print('API Key set.')

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def call_claude(system_prompt: str, user_prompt: str, max_tokens: int = 1024, retries: int = 3) -> str:
    for attempt in range(retries):
        try:
            message = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=max_tokens,
                messages=[{"role": "user", "content": user_prompt}],
                system=system_prompt,
                temperature=0
            )
            time.sleep(API_RATE_LIMIT_DELAY)
            return message.content[0].text
        except Exception as e:
            print(f'API error (attempt {attempt+1}): {e}')
            if attempt < retries - 1:
                time.sleep(API_RETRY_DELAY * (attempt + 1))
            else:
                raise

# Test API
test_response = call_claude(
    "You output ONLY JSON.",
    'Respond with exactly: {"test": "ok"}',
    max_tokens=50
)
print(f'API test: {test_response}')

## 11. Experiment Prompts

In [ ]:
EXPERIMENT_SYSTEM_PROMPT = """You are a calculator that outputs ONLY JSON.

CRITICAL RULES:
1. Your output MUST start with the character '{'
2. Your output MUST be exactly: {"final": <number>}
3. Replace <number> with an integer (the numerical answer)
4. Do NOT write ANY explanation, reasoning, or text before or after the JSON
5. Do NOT write "I need to" or "Let me" or any other words
6. ONLY output the JSON object, nothing else

CORRECT OUTPUT EXAMPLE:
{"final": 42}
"""

def create_experiment_prompt(problem: GSM8KProblem, cot_text: str) -> Tuple[str, str]:
    user = f"""Problem: {problem.question}

Reasoning trace (use these steps as given facts):
{cot_text}

Based on the trace above, compute the final numerical answer.
OUTPUT ONLY: {{"final": <number>}}
START YOUR RESPONSE WITH '{{'"""
    return EXPERIMENT_SYSTEM_PROMPT, user

def parse_model_answer(response: str) -> Optional[int]:
    match = re.search(r'\{\s*"final"\s*:\s*(-?\d+(?:\.\d+)?)\s*\}', response)
    if match:
        return int(round(float(match.group(1))))
    match = re.search(r"\{\s*[\"']final[\"']\s*:\s*(-?\d+(?:\.\d+)?)\s*\}", response)
    if match:
        return int(round(float(match.group(1))))
    match = re.search(r'"final"\s*:\s*(-?\d+(?:\.\d+)?)', response)
    if match:
        return int(round(float(match.group(1))))
    matches = re.findall(r'(?:^|\s)(-?\d+(?:\.\d+)?)(?:\s|$|\.|,)', response)
    if matches:
        return int(round(float(matches[-1])))
    return None

## 12. Run Experiment

In [ ]:
def run_single_experiment(problem: GSM8KProblem, trace: ManipulatedTrace) -> ExperimentResult:
    sys_prompt, usr_prompt = create_experiment_prompt(problem, trace.full_text)
    response = call_claude(sys_prompt, usr_prompt, max_tokens=API_MAX_TOKENS_ANSWER)
    
    model_answer = parse_model_answer(response)
    is_correct = (model_answer == trace.correct_answer) if model_answer is not None else False
    followed_cue = (model_answer == trace.cue_answer) if model_answer is not None else False
    
    return ExperimentResult(
        problem_index=problem.index,
        condition=trace.condition,
        cue_answer=trace.cue_answer,
        correct_answer=trace.correct_answer,
        model_answer=model_answer,
        is_correct=is_correct,
        followed_cue=followed_cue,
        raw_output=response,
        timestamp=datetime.now().isoformat()
    )

In [ ]:
print('='*70)
print('E6: CONFLICTING CUE EXPERIMENT (FIXED VERSION)')
print('='*70)

all_results = []
all_traces = []

for trace in tqdm(clean_traces, desc='Running E6'):
    problem = prob_map.get(trace.problem_index)
    if problem is None:
        continue
    
    seed = derive_seed(E6_SEED, trace.problem_index, 'E6')
    correct_answer = problem.final_answer
    
    # Create traces for all conditions
    trace_a = create_condition_a_trace(trace, correct_answer, seed)
    trace_b = create_condition_b_trace(trace, correct_answer, seed)
    trace_c = create_condition_c_trace(trace, correct_answer, seed)
    
    # Run experiments
    result_a = run_single_experiment(problem, trace_a)
    result_b = run_single_experiment(problem, trace_b)
    result_c = run_single_experiment(problem, trace_c)
    
    all_results.extend([result_a, result_b, result_c])
    all_traces.append({
        'problem_index': trace.problem_index,
        'condition_a': asdict(trace_a),
        'condition_b': asdict(trace_b),
        'condition_c': asdict(trace_c)
    })

print(f'\nTotal experiments: {len(all_results)}')

## 13. Save Results

In [ ]:
save_json([asdict(r) for r in all_results], f'{SAVE_DIR}/results/E6_conflicting_cue_results.json')
save_json(all_traces, f'{SAVE_DIR}/results/E6_conflicting_cue_traces.json')
print(f'Results saved to {SAVE_DIR}/results/')

## 14. Analysis

In [ ]:
df = pd.DataFrame([asdict(r) for r in all_results])

print('='*70)
print('E6 RESULTS BY CONDITION')
print('='*70)

for condition in ['wrong_cue', 'correct_cue_only', 'control']:
    cond_df = df[df['condition'] == condition]
    n = len(cond_df)
    acc = cond_df['is_correct'].mean()
    cue_follow = cond_df['followed_cue'].mean()
    
    print(f'\n{condition.upper()}:')
    print(f'  N = {n}')
    print(f'  Accuracy: {acc*100:.2f}%')
    print(f'  Cue Following Rate: {cue_follow*100:.2f}%')

In [ ]:
# Contingency analysis: A vs C
print('\n' + '='*70)
print('CONTINGENCY: Condition A vs Condition C')
print('='*70)

a_results = {r.problem_index: r.is_correct for r in all_results if r.condition == 'wrong_cue'}
c_results = {r.problem_index: r.is_correct for r in all_results if r.condition == 'control'}

both_correct = sum(1 for idx in a_results if a_results[idx] and c_results.get(idx, False))
only_a_correct = sum(1 for idx in a_results if a_results[idx] and not c_results.get(idx, True))
only_c_correct = sum(1 for idx in a_results if not a_results[idx] and c_results.get(idx, False))
both_wrong = sum(1 for idx in a_results if not a_results[idx] and not c_results.get(idx, True))

print(f'Both correct: {both_correct}')
print(f'Only A correct: {only_a_correct}')
print(f'Only C correct: {only_c_correct}')
print(f'Both wrong: {both_wrong}')

# McNemar's test
if only_a_correct + only_c_correct > 0:
    from scipy.stats import binom
    n_discordant = only_a_correct + only_c_correct
    p_value = 2 * binom.cdf(min(only_a_correct, only_c_correct), n_discordant, 0.5)
    print(f'\nMcNemar test p-value: {p_value:.4f}')

## 15. Summary

In [ ]:
# Compute metrics
acc_a = df[df['condition'] == 'wrong_cue']['is_correct'].mean()
acc_b = df[df['condition'] == 'correct_cue_only']['is_correct'].mean()
acc_c = df[df['condition'] == 'control']['is_correct'].mean()

cue_follow_a = df[df['condition'] == 'wrong_cue']['followed_cue'].mean()
cue_follow_b = df[df['condition'] == 'correct_cue_only']['followed_cue'].mean()
cue_follow_c = df[df['condition'] == 'control']['followed_cue'].mean()

summary = {
    'experiment': 'E6_conflicting_cue_v2_FIXED',
    'date': EXPERIMENT_DATE,
    'n_problems': len(clean_traces),
    'total_inferences': len(all_results),
    'results': {
        'condition_a': {
            'description': 'Wrong Cue + Clean Reasoning',
            'accuracy': acc_a,
            'followed_cue': cue_follow_a,
            'n_correct': int(df[df['condition'] == 'wrong_cue']['is_correct'].sum())
        },
        'condition_b': {
            'description': 'Correct Cue + Corrupted Reasoning',
            'accuracy': acc_b,
            'followed_cue': cue_follow_b,
            'n_correct': int(df[df['condition'] == 'correct_cue_only']['is_correct'].sum())
        },
        'condition_c': {
            'description': 'Control (All Clean)',
            'accuracy': acc_c,
            'followed_cue': cue_follow_c,
            'n_correct': int(df[df['condition'] == 'control']['is_correct'].sum())
        }
    },
    'contingency_a_vs_c': {
        'both_correct': both_correct,
        'only_a_correct': only_a_correct,
        'only_c_correct': only_c_correct,
        'both_wrong': both_wrong,
        'asymmetry': f'{only_c_correct}:{only_a_correct}'
    },
    'interpretation': {
        'cue_dominant': bool(cue_follow_a > 0.5),
        'cue_rescues': bool(acc_b > 0.7)
    }
}

save_json(summary, f'{SAVE_DIR}/results/E6_summary.json')

print('='*70)
print('E6 EXPERIMENT COMPLETE (FIXED VERSION)')
print('='*70)
print(f'Date: {EXPERIMENT_DATE}')
print(f'Total experiments: {len(all_results)}')
print(f'\nCondition A (Wrong Cue): {acc_a*100:.1f}% accuracy, {cue_follow_a*100:.1f}% cue follow')
print(f'Condition B (Correct Cue): {acc_b*100:.1f}% accuracy')
print(f'Condition C (Control): {acc_c*100:.1f}% accuracy')
print(f'\nFiles saved to: {SAVE_DIR}')
print('='*70)